In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [18]:
location= 'Data/properatti.csv'

df = pd.read_csv(location, index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '../Workshop/properatti.csv'

In [4]:
# Filtro sólo CABA y elmino los stores del tipo de propiedad
df2 = df[(df['property_type'].isin(['apartment','house','PH'])) & (df['state_name'] == 'Capital Federal')]

# Chequeo NaN
display(df2.info())

# Saco las columnas operatcion y estado, ya que son columnas con un solo valor y sin NA
df2 = df2.drop(['properati_url','expenses','operation', 'state_name', 'lat','lon','place_with_parent_names', 'country_name', 'geonames_id','price','currency','price_aprox_local_currency'], axis=1)

NameError: name 'df' is not defined

In [5]:
df2['place_name'].value_counts()

NameError: name 'df2' is not defined

In [6]:
df2.columns

NameError: name 'df2' is not defined

## Columnas a agregar/eliminar

- Amenities: Booleano. Pileta, garage/cochera, parrilla, sum, garden/jardin
- Cuota: Booleano. Ver si sigue aplicando para planes, y si eliminamos o no.
- superficie: unificar los regex de los dos grupos
- Sup total y cubierta: Tomas buscará en los trabajos del resto. Julio había imputado esto, traer el código al nuevo dataset. Revisar le tema de las que tiene valores diferentes.
- image_thumbnail: Ver si los duplicados que se encontraron son relevantes en el nuevo dataset, y ulego eliminarla
- Floor: Aplicar el regex y ver con cuantos datos nos quedamos.
- rooms y ambientes: Ver los regex que se armaron, como aplican al nuevo dataset
- expensas por ahora se elimina
- Description y title: Se saca todos los datos y se elimina

In [9]:
#regex para encontrar las diferentes características

patron = '(?P<m2>\d\d\d+\s\s*m2|\d\d\d+\s\s*M2|\d\d\d+\s\s*metro|\d\d\d+\s\s*METRO)|(?P<ambiente>\d+\sambiente|\d+\sAMBIENTE|\d+\sAMB|\d+\samb|\d+\sAMBS|\d+\sambs)|(?P<balcon>balcon|BALCON)|(?P<planta_alta>planta\salta|PLANTA\sALTA)|(?P<patio>patio|PATIO)|(?P<duplex>duplex|DUPLEX)|(?P<cuota>cuota|CUOTA)|(?P<amenities>parrilla|bbq|barbaque|asado|asador|cochera|pileta|piscina|pile|patio|terraza|solarium|pile|jardin|garden|green|parque|parquizado|pasto)+|\s(sum)+'
patron_regex = re.compile(patron)
patron_match = df2.description.apply(lambda x : patron_regex.search(str(x)))

# Extrae los metros cuadrados de la descripción, luego extrae el número (sin unidad) y posteriormente elimina aquellos menores que 3 para que no tome por error el número 2 de la unidad de medida
# Genera una serie m2

m2 = patron_match.apply(lambda x: x if x is None else x.group('m2'))
patron_num = '\d+'
patron_num_regex = re.compile(patron_num)
m2 = m2.apply(lambda x: x if x is None else patron_num_regex.findall(x))
# Extrae solo el valor de la superficie, evitando considerar el 2 de "m2"
m2 = m2.apply(lambda x: x if x is None else x[0])
m2 = pd.to_numeric(m2)
m2 = m2.apply(lambda x: x if x is None else (None if x<3 else x))

# Extrae la cantidad de ambientes, de la descripción, luego extrae el número (sin unidad) y posteriormente elimina aquellos menores que 3 para que no tome por error el número 2 de la unidad de medida
# Genera una serie ambientes

ambientes = patron_match.apply(lambda x: x if x is None else x.group('ambiente'))
ambientes = ambientes.apply(lambda x: x if x is None else x[0])
ambientes = pd.to_numeric(ambientes)


# Extrae la cantidad de ambientes, balcón, patio, duplex, y planta alta y amenities de la descripción. Genera una serie booleana

balcon = patron_match.apply(lambda x: x if x is None else x.group('balcon'))
balcon = balcon.apply(lambda x: False if x is None else True)

patio = patron_match.apply(lambda x: x if x is None else x.group('patio'))
patio = patio.apply(lambda x: False if x is None else True)

duplex = patron_match.apply(lambda x: x if x is None else x.group('duplex'))
duplex = duplex.apply(lambda x: False if x is None else True)

planta_alta = patron_match.apply(lambda x: x if x is None else x.group('planta_alta'))
planta_alta = planta_alta.apply(lambda x: False if x is None else True)

amenities = patron_match.apply(lambda x: x if x is None else x.group('amenities'))
amenities = amenities.apply(lambda x: False if x is None else True)

cuota = patron_match.apply(lambda x: x if x is None else x.group('cuota'))
cuota = cuota.apply(lambda x: False if x is None else True)

NameError: name 'df2' is not defined

In [10]:
# Se agregan las columnas a df2

df2['sup_m2_re'] = m2
df2['ambientes'] = ambientes
df2['balcon'] = balcon
df2['patio'] = patio
df2['duplex'] = duplex
df2['planta_alta'] = planta_alta
df2['amenities'] = amenities
df2['cuota'] = amenities

NameError: name 'm2' is not defined

In [11]:
mask_sup_cov = (df2['sup_m2_re'].notna()) & (df2['surface_covered_in_m2'].isna())
mask_sup_cov.sum()

NameError: name 'df2' is not defined

In [12]:
# Función que completa datos faltantes de superficie con datos extraídos de la descripción.
def compl_m2_re (row):
    if row['surface_covered_in_m2']>=0:
        result = row['surface_covered_in_m2']
    else:
        result = row['sup_m2_re']
    return result

df2['surface_covered_in_m2'] = df2.apply(compl_m2_re, axis = 1)

NameError: name 'df2' is not defined

In [13]:
# Verifico si se imputaron correctamente.

df2['surface_covered_in_m2'].notnull().sum()

NameError: name 'df2' is not defined

In [14]:
# Determinar la cantidad de filas que pueden tener invertidas los datos de superficie cubierta con superficie total

sup_mask = df2['surface_covered_in_m2'] > df2['surface_total_in_m2']
print(f'Cantidad de filas con sup_cov > sup_total {sup_mask.sum()}')

NameError: name 'df2' is not defined

In [15]:
# Función para corregir los valores invertidos de superficie cubierta y superficie total

df2['surface_covered_in_m2_aux'] = df2['surface_covered_in_m2']

def invertir_sup_cov (row):
    if (row['surface_covered_in_m2'] > row['surface_total_in_m2']):
        return row['surface_total_in_m2']
    else:
        return row['surface_covered_in_m2']

def invertir_sup_tot (row):
    if (row['surface_covered_in_m2'] == row['surface_total_in_m2']):
        return row['surface_covered_in_m2_aux']
    else:
        return row['surface_total_in_m2'] 
    
df2.loc[sup_mask,'surface_covered_in_m2'] = df2.apply(invertir_sup_tot, axis=1)
df2.loc[sup_mask,'surface_total_in_m2']  = df2.apply(invertir_sup_tot, axis=1)

NameError: name 'df2' is not defined

In [16]:
# Elimino la columna auxiliar generada

df2.drop('surface_covered_in_m2_aux', axis=1, inplace=True)

NameError: name 'df2' is not defined

In [17]:
# Verificación de la corrección de cambio de columnas e imputación.

sup_mask = df2['surface_covered_in_m2'] > df2['surface_total_in_m2']
print(f'Cantidad de filas con sup_cov > sup_total {sup_mask.sum()}')
display(df2.info())

NameError: name 'df2' is not defined

## Feature engineering

## test modelado básico

In [43]:
# importo los librerías para el modelo
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn import metrics

In [25]:
#Dropeo todas las features que necesitan tratamiento previo para servir
df3 = df2.drop(['description','title','image_thumbnail','price_per_m2','lat-lon', 'rooms','floor'], axis=1)

In [26]:
mask_barrios = (df3['place_name'].isin(['Belgrano','Caballito']))
mask_appt = (df3['property_type'] == 'apartment')

df3 = df3[mask_barrios & mask_appt]

In [27]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4840 entries, 7 to 121215
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   property_type          4840 non-null   object 
 1   place_name             4840 non-null   object 
 2   price_aprox_usd        4224 non-null   float64
 3   surface_total_in_m2    4119 non-null   float64
 4   surface_covered_in_m2  4643 non-null   float64
 5   price_usd_per_m2       3530 non-null   float64
dtypes: float64(4), object(2)
memory usage: 264.7+ KB


In [28]:
#Descarto TODAS las que tienen algun NaN, sin otro criterio
df3 = df3.dropna()

In [46]:
df3['place_name'].value_counts()

Belgrano     1874
Caballito    1528
Name: place_name, dtype: int64

In [35]:
# Selecciono las columnas que considero features para modelar 
X = df3.loc[:,['place_name','surface_total_in_m2','surface_covered_in_m2']]
y = df3['price_usd_per_m2']

# Convierto a dummies las categóricas
X = pd.get_dummies(X,drop_first=True)

# Separo en train/test
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=33)

X.shape, y.shape, X_train.shape,X_test.shape, y_train.shape, y_test.shape

((3402, 3), (3402,), (2551, 3), (851, 3), (2551,), (851,))

In [38]:
lr = LinearRegression()

lr.fit(X_train,y_train)
lr.intercept_, lr.coef_

(3031.862519234963, array([  -9.35812977,   12.63713738, -600.1193796 ]))

In [44]:
y_pred = lr.predict(X_test)

metrics.r2_score(y_test, y_pred)

0.27414053361027124

In [52]:
#prueba con crossvalidation con Ridge
rcv = RidgeCV(alphas=(0.0001,0.001,0.01,0.1,1,10,100,1000,10000), cv=5)

rcv.fit(X_train,y_train)
rcv.intercept_, rcv.coef_, rcv.alpha_

(3031.8624621393838,
 array([  -9.35812972,   12.63713749, -600.11928047]),
 0.0001)